<a href="https://colab.research.google.com/github/varshachawan/NaturalLanguageProcessing/blob/master/service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
import timeit

from flask import Flask, jsonify, request, make_response



In [0]:
def load_models():
  model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
  tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
  model = model_class.from_pretrained(pretrained_weights)
  loaded_model = joblib.load('SavedModel.joblib')



In [0]:
def create_embeding(text):
  x = 'I hated the movie very much'
  tokenized=tokenizer.encode(text, add_special_tokens=True)
  max_len=60
  padded = np.array([tokenized + [0]*(max_len-len(tokenized))])
  attention_mask = np.where(padded != 0, 1, 0)
  attention_mask.shape
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
    features = last_hidden_states[0][:,0,:].numpy()
  return features


In [0]:
def predict(input_text):
  feature_vector =create_embeding(input_text)
  pred=loaded_model.predict(feature_vector)
  if pred==0:
    return 'negative'
  else:
    return 'positive'


In [0]:
app = Flask(__name__)


In [0]:
@app.route('/predict', methods=['get'])
def predict_sentiment():
    text = request.get('text')
    print(text)
    predict = predict_fn(text)

    return jsonify({'predictions': predict})


In [14]:
if __name__ == '__main__':
    load_models()
    app.run(host='0.0.0.0', port='5000')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


curl: (7) Failed to connect to 0.0.0.0 port 5000: Connection refused
